In [ ]:
k_shot = 0
explain = False
save_dir = "."
dataset = "copa"

# Disclaimer: This GPT-3 code is to demonstrate the logic of running and obtaining results from GPT-3.
# This notebook and related code were written *after* the result files .bin were obtained.
# We are unable to test this code as we ran out of OpenAI free credits :(
# Nevertheless, if one can run this notebook without errors, one should get very close results to the report,
# modulo the randomness in GPT-3 generation results.

## Imports

In [ ]:
import datasets
import time
import numpy as np
import zipfile
import pandas as pd
import pickle
from run_gpt3 import prepare_copa, prepare_ecare, get_prompts_with_labels, save_results, get_gpt3_prediction
import openai

## Setups

In [ ]:
assert explain == False or dataset == "ecare", "COPA does not have explanations"

openai.api_key_path = "/path/to/text/file/with/openai/api/key"

## Run

In [ ]:
if dataset == "copa":
    train_set, dev_set = prepare_copa()
elif dataset == "ecare":
    train_set, dev_set = prepare_ecare()
prompts = get_prompts_with_labels(train_set, dev_set, k_shot, explain)
gpt_preds = []
curr_prompt_idx = 0
prompts_submitted = {k: False for k in range(prompts.num_rows)}


In [ ]:
# this cell can be interruppted and resumed arbitrary

print(f"Started running from example #{curr_prompt_idx}")
while True:
    try:
        if curr_prompt_idx == prompts.num_rows:
            print("Finished.")
            break
        if not prompts_submitted[curr_prompt_idx]:
            prompt = prompts[curr_prompt_idx]["prompt"]
            pred = get_gpt3_prediction(prompt)
            gpt_preds.append(pred)
            prompts_submitted[curr_prompt_idx] = True
        curr_prompt_idx += 1
    except openai.error.RateLimitError:
        print(f"Sleeping at example #{curr_prompt_idx}.")
        time.sleep(60)
        continue
    except KeyboardInterrupt:
        print(f"Interrupted at example #{curr_prompt_idx}. Pausing.")
        break

In [ ]:
save_results(gpt_preds, prompts, k_shot=k_shot, dataset=dataset, explain=explain, save_dir=save_dir)